<a href="https://colab.research.google.com/github/narendra974/GAN/blob/main/vae_auto_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras.datasets import mnist
import numpy as np


In [ ]:
batch_size = 100
original_dim = 28*28
latent_dim = 2
intermediate_dim = 256
nb_epoch = 5
epsilon_std = 1.0

In [ ]:
def Sampling(args):
  z_mean, z_log_var  = args
  epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0)
  return z_mean + K.exp(z_log_var/2) * epsilon


x = Input(shape=original_dim, name="input")
h = Dense(intermediate_dim, activation='relu', name='encoding')(x)
z_mean = Dense(latent_dim, name='mean')(h)
z_variance = Dense(latent_dim, name='log-variance')(h)
z=Lambda(Sampling, output_shape=(latent_dim, ))([z_mean, z_variance])
encoder = Model(x, [z_mean, z_variance, z], name="encoder")

In [ ]:
input_decoder = Input(shape=(latent_dim, ), name='decoder_input')
decoder_h = Dense(intermediate_dim, activation='relu', name='decoder_h')(input_decoder)
x_decoded = Dense(original_dim, activation='sigmoid', name='flat_decoded')(decoder_h)
decoder = Model(input_decoder, x_decoded, name='decoder')

In [ ]:
def vae_loss(x, x_decoded_mean, z_log_var, x_mean, original_dim):